In [2]:
import numpy as np
import matplotlib.pyplot as plt

# Generate random data
np.random.seed(42)

Matplotlib is building the font cache; this may take a moment.


In [3]:
a_true, b_true = 2.5, -1.0
n_points = 100

x = np.random.uniform(-2,3,n_points)
y = a_true * x + b_true + np.random.normal(0, 0.5, n_points)

split_idx = int(0.8 * n_points)
x_train, x_test = x[:split_idx], x[split_idx:]
y_train, y_test = y[:split_idx], y[split_idx:]

In [4]:
# define model and loss function (evaluate model performance)
def predict (a, b, x):
    return a * x + b
def mse_loss (a, b, x, y):
    predictions = predict(a, b, x)
    residulas = y - predictions
    return np.mean(residulas**2)
true_loss = mse_loss(a_true, b_true, x_train, y_train)

print(f"MSE Loss: {true_loss}")

MSE Loss: 0.20321589412252677


In [11]:
# brute force grid search
a_grid = np.linspace (-1, 5, 8000) #201 values for slope
b_grid = np.linspace (-1,2,8000) #201 values for intercept

A_mesh, B_mesh = np.meshgrid(a_grid, b_grid)
loss_grid = np.zeros_like(A_mesh)



for i, a in enumerate(a_grid):
    for j, b in enumerate(b_grid):
        loss_grid[i,j] = mse_loss(a, b, x_train, y_train)

print("grid search complete:")

grid search complete:


In [12]:
min_idx = np. unravel_index(np.argmin(loss_grid), loss_grid.shape)
a_best = a_grid[min_idx[0]]
b_best = b_grid[min_idx[1]]
best_loss = loss_grid[min_idx]
print(f"Best parameters: a={a_best}, b={b_best}, MSE Loss={best_loss}")

Best parameters: a=2.435429428678585, b=-1.0, MSE Loss=0.19317803720640842
